In [1]:
import sys
import cv2
import pytesseract
from PIL import Image

import numpy as np

### OpenCV - 이미지 읽기, 쓰기 및 표시하기 (1)


In [2]:
def handle_image():
    imgfile = 'images/sample.png'
    # imread_color : 컬러 이미지로 불러오겠다!!
    img = cv2.imread(imgfile, cv2.IMREAD_COLOR)
    
    cv2.imshow('image', img)
    cv2.waitKey(0)  # 사용자가 키보드를 누를 때 까지 기둘.. 숫자는 시간을 나타냄, 0은 무한정 기둘..
    cv2.destroyAllWindows()  # 모든 윈도우 닫음
    cv2.waitKey(1) # 없어도 됨, 주피터에 버그 있어서 이렇게 추가해야 동작함
    
if __name__ == '__main__':
    handle_image()

---

### OpenCV - 이미지 읽기, 쓰기 및 표시하기 (2)


In [4]:
def handle_image():
    imgfile = 'images/sample.png'
    # imread_greyscale : 흑백 이미지로 불러오겠다!! flag 변화
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)  # 노말 : 원본 이미지 크기, 크기조절 가능
    cv2.imshow('image', img)
    cv2.waitKey(0)  
    
    k = cv2.waitKey(0)
    
    if k == 27:  # 27 - esc 키
        cv2.destroyAllWindows()  # 모든 윈도우 닫음
        cv2.waitKey(1) # 없어도 됨, 주피터에 버그 있어서 이렇게 추가해야 동작함

    elif k == ord('s'):  # s 키 누르면 저장!!
        cv2.imwrite('grayImage.png', img)  # (파일경로, 저장할 이미지 객체)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
if __name__ == '__main__':
    handle_image()

---

### # OpenCV - 도형 외곽 추출하기 (1)


In [1]:
from matplotlib import pyplot as plt

def contour():
    imgfile = 'images/contour.jpg'
    img = cv2.imread(imgfile)
    
    # cvcolor :색공간 추가
    # bgr2gray :rgb 영역에서 gray컬러로 바꾸겠다.
    # 엣지 디텍션은 이미지를 변형시키니까, 복사본 (grey)를 만든다.
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    # canny 알고리즘 : 엣지 찾는 것
    # 100 해당 값보다 작으면 엣지 아닌 것으로 봄
    # 200 해당 값보다 높아야만 엣지로 봄 - 적당한 값을 찾아야 해
    edge = cv2.Canny(imgray, 100,200)
    
    # findcontours : 외곽 찾는 함수, 흰색과 검은색으로 이루어진 이미지를 넘겨야해
    # retr_tree : 계층관계를 고려하여 리턴,
    # chain_app... : 
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.imshow('edge', edge)
    # 외곽 그리기 (이미지, 외곽 객체, 컨투어 인덱스(모두 그리고 싶으면 -1), BGR 색상, 선 두께)
    cv2.drawContours(img, contours, -1,(0,255,0),1)
    cv2.imshow('contour', img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour() 

NameError: name 'cv2' is not defined

---

### OpenCV - 도형 외곽 추출하기 (2)
- 문서(명함)이 조금 구겨져 있으면?
- 정확한 직사각형이 아니면?
    - 도형을 근사하여 계산함으로써 overfitting을 방지

In [11]:
def contour_approx():
    imgfile = 'images/contour2.png'
    img = cv2.imread(imgfile)
    img2 = img.copy()
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edge = cv2.Canny(imgray,100,200)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cnt = contours[0]
    cv2.drawContours(img, [cnt], 0, (0,255,0), 3)
    
    ################img2 (꼭지점을 4개로 근사)###################
    # 외곽 둘레길이 계산
    # 근사정확도 (오차)
    epsilon = 0.1 * cv2.arcLength(cnt, True)
    
    # 다각형을 대상으로 꼭지점을 줄여나가는 것
    # 오차값이 작을수록 원본과 비슷
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    
    cv2.drawContours(img2, [approx], 0, (0,255,0), 3)
    
    cv2.imshow('Contour', img)
    cv2.imshow('Approx', img2)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour_approx() 

---

### 투영변환 구현하기 (1)
- 외곽으로 그려진 영역을 반듯하게 변환하기

In [4]:
def warp_affine():
    img = cv2.imread('images/transform.png')
    
    # x,y 좌표 이동 (pts1 -> pts2 좌표로)
    pts1 = np.float32([[50,50],[200,50],[20,200]])
    pts2 = np.float32([[70,100],[220,50],[150,250]])
    
    # 좌표이동 매트릭스 getaffinetransform
    # 이동할 파라메터, 이동 된 파라메동
    M = cv2.getAffineTransform(pts1, pts2)
    
    # 픽셀 좌표이동 warpAffine
    result = cv2.warpAffine(img, M, (350,300))
    
    cv2.imshow('original', img)
    cv2.imshow('Affine Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    warp_affine()

### OpenCV - 투영변환 구현하기 (2)
- 포인트를 지정하지 않고 자동으로 반듯하게 하는 방법은?


In [10]:
def warp_perspective():
    img = cv2.imread('images/transform.jpg')
    
    #좌표
    # 외곽 검출법으로 네개의 꼭지점을 찾았다고 가정
    topLeft = [127,157]
    topRight = [448, 152]
    bottomRight = [579, 526]
    bottomLeft = [54, 549]
    
    pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
    
    
    # 넓이2 높이2 절대값 구하기
    w1 = abs(bottomRight[0] - bottomLeft[0])
    w2 = abs(topRight[0] - topLeft[0])
    h1 = abs(topRight[1] - bottomRight[1])
    h2 = abs(topLeft[1] - bottomLeft[1])
    
    # 최소넓이, 최대 높이
    minWidth = min([w1, w2])
    minHeight = min([h1, h2])
    
    # 변환 될 대상 좌표
    # 1을 뺀 이유 = 비어있는 픽셀을 만들지 않기 위해서
    
    pts2 = np.float32([[0,0], [minWidth-1,0], [minWidth-1,minHeight-1], [0,minHeight-1]])
    
    # getperspective : 좌표기반 픽셀 변환,원근 보정 고려함
    # affine : 좌표기반 픽셀 변환, 원근보정 하지 않음
    M = cv2.getPerspectiveTransform(pts1, pts2)
    
    result = cv2.warpPerspective(img, M, (int(minWidth), int(minHeight)))
    
    cv2.imshow('original', img)
    cv2.imshow('Warp Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    warp_perspective()   

---

### 스캔 한 듯한 효과 주기 (1)
- 조명의 영향 제거하기 (글자인식을 위해서)
- 모든 색을 0과 1 (흰, 검)로 바꿈 : Threshold

In [4]:
import numpy as np
import cv2

# Callback Function for Trackbar (but do not any work)
def nothing(x):
    pass


def global_threshold():
    imgfile = 'images/document.jpg'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    # resize image - 연산량 조절을 위해서
    r = 600.0 / img.shape[0]   # 이미지 가로 픽셀 600으로 고정
    dim = (int(img.shape[1] * r),600)  # 세로 길이
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    WindowName = 'Window'
    TrackbarName = 'Threshold'
    
    # MAke Window and Trackbar
    cv2.namedWindow(WindowName)
    cv2.createTrackbar(TrackbarName, WindowName, 70, 255, nothing)  # noting 바 이동을 하면 특정 행동 수행 (함수)
    
    # Allocate destination image
    Threshold = np.zeros(img.shape, np.uint8)  #shape : 이미지 가로 세로 픽셀
    
    # Loop for get trackbar pos and precess it
    while True:
        #get position in trackbar
        TrackbarPos = cv2.getTrackbarPos(TrackbarName, WindowName)
        
        #Apply threshold
        cv2.threshold(img, TrackbarPos, 255, cv2.THRESH_BINARY, Threshold) # 255 threshold 최대값
        
        # Show in window
        cv2.imshow(WindowName, Threshold)
        
        # wait for ESC key to exit
        k = cv2.waitKey(0)
        if k == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
    return

if __name__ == '__main__':
    global_threshold() 

---

### 스캔한 듯한 효과 주기 - 2
- Threshold를 자동으로 구할수는 없나?
- 조명의 영향을 조금 더 줄일 수 없을까?
- 이미지를 잘개 쪼개서 threshold를 구하자!!!


In [5]:
# OpenCV - 스캔한 듯한 효과 주기 (2)
# 배경 외곽 노이즈가 적을수록 모서리 추축하기가 쉬움
# blur 효과 때문에 결과적으로 글자는 흐릿해짐
# edge 검출 (blur)  ->  꼭지점 저장  ->  blur 안준 이미지를 사용 (글자인식 위해서) 

import numpy as np
import cv2

def adaptive_threshold():
    imgfile = 'images/document.jpg'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    # Resize image
    r = 600.0 / img.shape[0]
    dim = (int(img.shape[1] * r), 600)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # Blur image and apply adaptive threshold
    # 2진화 결과를 명확하게 하기 위해서 blur 효과를 준다.
    blur = cv2.GaussianBlur(img, (5, 5), 0)   # gaussianblur -> 이미지에 blur 흐리게 함 (이미지 대상, (주변 픽셀), ) 주변픽셀 커질수록 그림이 뭉개짐
    result_without_blur = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10) # 21 - 픽셀 잘게 쪼개는 정도
    result_with_blur = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10) # 10 - 주변 영역 평균값에서 빼는 상수값 10이 적당
    cv2.imshow('Without Blur', result_without_blur)
    cv2.imshow('With Blur', result_with_blur)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    adaptive_threshold() 

---

### 명함인식 구현 (각각 단계 합치기!!)
- 

In [3]:
# 명함인식 구현하기 - 캡처된 이미지

import numpy as np
import cv2

def order_points(pts):  # 꼭지점 반환
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

# edge detection!!
def auto_scan_image():
    # load the image and compute the ratio of the old height
    # to the new height, clone it, and resize it
    # document.jpg ~ docuemnt7.jpg
    image = cv2.imread('images/document5.jpg')
    orig = image.copy()
    r = 800.0 / image.shape[0]
    dim = (int(image.shape[1] * r), 800)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image
    print ("STEP 1: Edge Detection")
    cv2.imshow("Image", image)
    cv2.imshow("Edged", edged)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # find the contours in the edged image, keeping only the, 계층관계 쓰지 않음 컨두어만 받음
    # largest ones, and initialize the screen contour
    (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5] # 컨투어 그린 면적을 큰 순서로 5개 가져옴

    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)  # 컨투어 길이 반환
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)  # 길이에 0.02 오차로 근사하여 외곽 추출

        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:   # 꼭지점이 4개라면, 명함의 외곽이라 봄
            screenCnt = approx
            break

    # show the contour (outline) of the piece of paper
    print ("STEP 2: Find contours of paper")
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)   # 컨투어 그림!!
    cv2.imshow("Outline", image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # apply the four point transform to obtain a top-down
    # view of the original image
    # 검출된 외곽으로, 반듯한 사각형 변환!!
    rect = order_points(screenCnt.reshape(4, 2) / r)  # 컨투어에서 4개 꼭지점 정렬 함수
    (topLeft, topRight, bottomRight, bottomLeft) = rect
    
    w1 = abs(bottomRight[0] - bottomLeft[0])
    w2 = abs(topRight[0] - topLeft[0])
    h1 = abs(topRight[1] - bottomRight[1])
    h2 = abs(topLeft[1] - bottomLeft[1])
    maxWidth = max([w1, w2])
    maxHeight = max([h1, h2])
    
    dst = np.float32([[0,0], [maxWidth-1,0], 
                      [maxWidth-1,maxHeight-1], [0,maxHeight-1]])
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight))

    # show the original and scanned images
    print ("STEP 3: Apply perspective transform")
    cv2.imshow("Warped", warped)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    # convert the warped image to grayscale, then threshold it
    # to give it that 'black and white' paper effect
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

    # show the original and scanned images
    print ("STEP 4: Apply Adaptive Threshold")
    cv2.imshow("Original", orig)
    cv2.imshow("Scanned", warped)
    cv2.imwrite('scannedImage.png', warped)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    auto_scan_image()

STEP 1: Edge Detection
STEP 2: Find contours of paper
STEP 3: Apply perspective transform
STEP 4: Apply Adaptive Threshold


---

### 명함인식 구현하기 - 웹캠 (1)
- 카메라에서 실시간으로 명함 인식.
- 루프를 돌면서 카메라에서 프레임을 추출한 후 알고리즘 적용

In [ ]:
# 명함인식 구현하기 - 웹캠(1)

import numpy as np
import cv2

def auto_scan_image_via_webcam():
    
    try: 
        cap = cv2.VideoCapture(0)
    except:
        print ('cannot load camera!')
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print ('cannot load camera!')
            break
            
        k = cv2.waitKey(10)
        if k == 27:
            break
        
        # convert the image to grayscale, blur it, and find edges
        # in the image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        edged = cv2.Canny(gray, 75, 200)

        # show the original image and the edge detected image
        print ("STEP 1: Edge Detection")

        # find the contours in the edged image, keeping only the
        # largest ones, and initialize the screen contour
        (_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

        # loop over the contours
        for c in cnts:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []

            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            # 불필요한 사각형을 검출하지 않는 로직!! 전체화면의 10% 이상 차지할 때만 사각형 검출
            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]       
                ratio = contourSize / camSize
                print (contourSize)
                print (camSize)
                print (ratio)
                
                if ratio > 0.1:
                    screenCnt = approx
                    
                break 
        
        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue
            
        else:
            # show the contour (outline) of the piece of paper
            print ("STEP 2: Find contours of paper")

            cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
            cv2.imshow("WebCam", frame)
        
    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    auto_scan_image_via_webcam()

---

### 웹캠에서 검출 - 스캔효과까지..

In [ ]:
# 명함인식 구현하기 - 웹캠(2)

import numpy as np
import cv2

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def auto_scan_image_via_webcam():
    
    try: 
        cap = cv2.VideoCapture(0)
    except:
        print ('cannot load camera!')
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print ('cannot load camera!')
            break
            
        k = cv2.waitKey(10)
        if k == 27:
            break

        # convert the image to grayscale, blur it, and find edges
        # in the image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        edged = cv2.Canny(gray, 75, 200)

        # show the original image and the edge detected image
        print ("STEP 1: Edge Detection")

        # find the contours in the edged image, keeping only the
        # largest ones, and initialize the screen contour
        #(_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

        # loop over the contours
        for c in cnts:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []

            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]
                ratio = contourSize / camSize
                print (contourSize)
                print (camSize)
                print (ratio)
                
                if ratio > 0.1:
                    screenCnt = approx
                    
                break 
        
        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue
            
        else:
            # show the contour (outline) of the piece of paper
            print ("STEP 2: Find contours of paper")

            cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
            cv2.imshow("WebCam", frame)
            
            # apply the four point transform to obtain a top-down
            # view of the original image
            rect = order_points(screenCnt.reshape(4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = rect

            w1 = abs(bottomRight[0] - bottomLeft[0])
            w2 = abs(topRight[0] - topLeft[0])
            h1 = abs(topRight[1] - bottomRight[1])
            h2 = abs(topLeft[1] - bottomLeft[1])
            maxWidth = max([w1, w2])
            maxHeight = max([h1, h2])

            dst = np.float32([[0,0], [maxWidth-1,0], 
                              [maxWidth-1,maxHeight-1], [0,maxHeight-1]])

            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))

            # show the original and scanned images
            print ("STEP 3: Apply perspective transform")

            # convert the warped image to grayscale, then threshold it
            # to give it that 'black and white' paper effect
            warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

            # show the original and scanned images
            print ("STEP 4: Apply Adaptive Threshold")

            break
        
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    cv2.imshow("Scanned", warped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
        
    
if __name__ == '__main__':
    auto_scan_image_via_webcam()

---

### OCR - Tesseract

In [12]:
# OCR - Tesseract

from PIL import Image
import pytesseract

def ocr_tesseract():
    image_file = 'images/scannedImage.png'
    im = Image.open(image_file)
    text = pytesseract.image_to_string(im)
    im.show()

    print (text)

if __name__ == '__main__':
    ocr_tesseract()

vpstg OAR O
Special Coupon

“ PARKSEUNGCHOL
HAIRSTUDIO


---

### OCR - Project Oxford by MS

In [ ]:
# OCR - Project Oxford by MS

from PIL import Image
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

def print_text(json_data):
    result = json.loads(json_data)
    for l in result['regions']:
        for w in l['lines']:
            line = []
            for r in w['words']:
                line.append(r['text'])
            print (' '.join(line))
    return

def ocr_project_oxford(headers, params, data):
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/vision/v1.0/ocr?%s" % params, data, headers)
    response = conn.getresponse()
    data = response.read().decode()
    print (data + "\n")
    print_text(data)
    conn.close()
    return
    
if __name__ == '__main__':
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': '',  # free key 입력!!!
    }
    params = urllib.parse.urlencode({
        # Request parameters
        'language': 'unk',  # ko  한글 인식 가능하게..
        'detectOrientation ': 'true',
    })
    data = open('images/scannedImage.png', 'rb').read()
    
    try:
        image_file = 'images/scannedImage.png'
        im = Image.open(image_file)
        im.show()
        ocr_project_oxford(headers, params, data)
    except Exception as e:
        print (e)

---

### 최종 명함인식 - 웹캠 + OCR

In [ ]:
# 명함인식 구현하기 - 웹캠 + OCR

import numpy as np
import cv2
from PIL import Image
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

def print_text(json_data):
    result = json.loads(json_data)
    for l in result['regions']:
        for w in l['lines']:
            line = []
            for r in w['words']:
                line.append(r['text'])
            print (' '.join(line))
    return

def ocr_project_oxford(headers, params, data):
    conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
    conn.request("POST", "/vision/v1.0/ocr?%s" % params, data, headers)
    response = conn.getresponse()
    data = response.read().decode()
    print (data + "\n")
    print_text(data)
    conn.close()
    return

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def auto_scan_image_via_webcam():
    
    try: 
        cap = cv2.VideoCapture(0)
    except:
        print ('cannot load camera')
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print ('cannot load camera!')
            break
            
        k = cv2.waitKey(10)
        if k == 27:
            break

        # convert the image to grayscale, blur it, and find edges
        # in the image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        edged = cv2.Canny(gray, 75, 200)

        # show the original image and the edge detected image
        # print ("STEP 1: Edge Detection")

        # find the contours in the edged image, keeping only the
        # largest ones, and initialize the screen contour
        #(_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

        # loop over the contours
        for c in cnts:
            # approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []

            # if our approximated contour has four points, then we
            # can assume that we have found our screen
            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]
                ratio = contourSize / camSize
                # print (contourSize)
                # print (camSize)
                # print (ratio)
                
                if ratio > 0.1:
                    screenCnt = approx
                    
                break 
        
        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue
            
        else:
            # show the contour (outline) of the piece of paper
            print ("STEP 2: Find contours of paper")

            cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
            cv2.imshow("WebCam", frame)
            
            # apply the four point transform to obtain a top-down
            # view of the original image
            rect = order_points(screenCnt.reshape(4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = rect

            w1 = abs(bottomRight[0] - bottomLeft[0])
            w2 = abs(topRight[0] - topLeft[0])
            h1 = abs(topRight[1] - bottomRight[1])
            h2 = abs(topLeft[1] - bottomLeft[1])
            maxWidth = max([w1, w2])
            maxHeight = max([h1, h2])

            dst = np.float32([[0,0], [maxWidth-1,0], 
                              [maxWidth-1,maxHeight-1], [0,maxHeight-1]])

            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))

            # show the original and scanned images
            print ("STEP 3: Apply perspective transform")

            # convert the warped image to grayscale, then threshold it
            # to give it that 'black and white' paper effect
            warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
            warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

            # show the original and scanned images
            print ("STEP 4: Apply Adaptive Threshold")

            break
        
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    cv2.imshow("Scanned", warped)
    cv2.imwrite('scannedImage.png', warped)
    
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': '',
    }
    params = urllib.parse.urlencode({
        # Request parameters
        'language': 'unk',
        'detectOrientation ': 'true',
    })
    data = open('scannedImage.png', 'rb').read()
    
    try:
        image_file = 'scannedImage.png'
        ocr_project_oxford(headers, params, data)
    except Exception as e:
        print (e)
        
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
        
    
if __name__ == '__main__':
    auto_scan_image_via_webcam()

STEP 2: Find contours of paper
STEP 3: Apply perspective transform
STEP 4: Apply Adaptive Threshold
{"error":{"code":"401","message":"Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource."}}

'regions'


---

### OpenCV - 이미지에서 텍스트 영역만 찾아내기


In [14]:
# (참고) OpenCV - 이미지에서 텍스트 영역만 찾아내기

# 출처: http://www.danvk.org/2015/01/07/finding-blocks-of-text-in-an-image-using-python-opencv-and-numpy.html

import glob
import os
import random
import sys
import random
import math
import json
from collections import defaultdict

import cv2
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage.filters import rank_filter


def dilate(ary, N, iterations):
    """Dilate using an NxN '+' sign shape. ary is np.uint8."""
    kernel = np.zeros((N,N), dtype=np.uint8)
    kernel[(N-1)/2,:] = 1
    dilated_image = cv2.dilate(ary / 255, kernel, iterations=iterations)

    kernel = np.zeros((N,N), dtype=np.uint8)
    kernel[:,(N-1)/2] = 1
    dilated_image = cv2.dilate(dilated_image, kernel, iterations=iterations)
    dilated_image = cv2.convertScaleAbs(dilated_image)
    return dilated_image


def props_for_contours(contours, ary):
    """Calculate bounding box & the number of set pixels for each contour."""
    c_info = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        c_im = np.zeros(ary.shape)
        cv2.drawContours(c_im, [c], 0, 255, -1)
        c_info.append({
            'x1': x,
            'y1': y,
            'x2': x + w - 1,
            'y2': y + h - 1,
            'sum': np.sum(ary * (c_im > 0))/255
        })
    return c_info


def union_crops(crop1, crop2):
    """Union two (x1, y1, x2, y2) rects."""
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return min(x11, x12), min(y11, y12), max(x21, x22), max(y21, y22)


def intersect_crops(crop1, crop2):
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return max(x11, x12), max(y11, y12), min(x21, x22), min(y21, y22)


def crop_area(crop):
    x1, y1, x2, y2 = crop
    return max(0, x2 - x1) * max(0, y2 - y1)


def find_border_components(contours, ary):
    borders = []
    area = ary.shape[0] * ary.shape[1]
    for i, c in enumerate(contours):
        x,y,w,h = cv2.boundingRect(c)
        if w * h > 0.5 * area:
            borders.append((i, x, y, x + w - 1, y + h - 1))
    return borders


def angle_from_right(deg):
    return min(deg % 90, 90 - (deg % 90))


def remove_border(contour, ary):
    """Remove everything outside a border contour."""
    # Use a rotated rectangle (should be a good approximation of a border).
    # If it's far from a right angle, it's probably two sides of a border and
    # we should use the bounding box instead.
    c_im = np.zeros(ary.shape)
    r = cv2.minAreaRect(contour)
    degs = r[2]
    if angle_from_right(degs) <= 10.0:
        box = cv2.boxPoints(r)
        box = np.int0(box)
        cv2.drawContours(c_im, [box], 0, 255, -1)
        cv2.drawContours(c_im, [box], 0, 0, 4)
    else:
        x1, y1, x2, y2 = cv2.boundingRect(contour)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 255, -1)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 0, 4)

    return np.minimum(c_im, ary)


def find_components(edges, max_components=16):
    """Dilate the image until there are just a few connected components.
    Returns contours for these components."""
    # Perform increasingly aggressive dilation until there are just a few
    # connected components.
    count = 21
    dilation = 5
    n = 1
    while count > 16:
        n += 1
        dilated_image = dilate(edges, N=3, iterations=n)
        _, contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        count = len(contours)
    #print dilation
    #Image.fromarray(edges).show()
    #Image.fromarray(255 * dilated_image).show()
    return contours


def find_optimal_components_subset(contours, edges):
    """Find a crop which strikes a good balance of coverage/compactness.
    Returns an (x1, y1, x2, y2) tuple.
    """
    c_info = props_for_contours(contours, edges)
    c_info.sort(key=lambda x: -x['sum'])
    total = np.sum(edges) / 255
    area = edges.shape[0] * edges.shape[1]

    c = c_info[0]
    del c_info[0]
    this_crop = c['x1'], c['y1'], c['x2'], c['y2']
    crop = this_crop
    covered_sum = c['sum']

    while covered_sum < total:
        changed = False
        recall = 1.0 * covered_sum / total
        prec = 1 - 1.0 * crop_area(crop) / area
        f1 = 2 * (prec * recall / (prec + recall))
        #print '----'
        for i, c in enumerate(c_info):
            this_crop = c['x1'], c['y1'], c['x2'], c['y2']
            new_crop = union_crops(crop, this_crop)
            new_sum = covered_sum + c['sum']
            new_recall = 1.0 * new_sum / total
            new_prec = 1 - 1.0 * crop_area(new_crop) / area
            new_f1 = 2 * new_prec * new_recall / (new_prec + new_recall)

            # Add this crop if it improves f1 score,
            # _or_ it adds 25% of the remaining pixels for <15% crop expansion.
            # ^^^ very ad-hoc! make this smoother
            remaining_frac = c['sum'] / (total - covered_sum)
            new_area_frac = 1.0 * crop_area(new_crop) / crop_area(crop) - 1
            if new_f1 > f1 or (
                    remaining_frac > 0.25 and new_area_frac < 0.15):
                print('%d %s -> %s / %s (%s), %s -> %s / %s (%s), %s -> %s' % (
                        i, covered_sum, new_sum, total, remaining_frac,
                        crop_area(crop), crop_area(new_crop), area, new_area_frac,
                        f1, new_f1))
                crop = new_crop
                covered_sum = new_sum
                del c_info[i]
                changed = True
                break

        if not changed:
            break

    return crop


def pad_crop(crop, contours, edges, border_contour, pad_px=15):
    """Slightly expand the crop to get full contours.
    This will expand to include any contours it currently intersects, but will
    not expand past a border.
    """
    bx1, by1, bx2, by2 = 0, 0, edges.shape[0], edges.shape[1]
    if border_contour is not None and len(border_contour) > 0:
        c = props_for_contours([border_contour], edges)[0]
        bx1, by1, bx2, by2 = c['x1'] + 5, c['y1'] + 5, c['x2'] - 5, c['y2'] - 5

    def crop_in_border(crop):
        x1, y1, x2, y2 = crop
        x1 = max(x1 - pad_px, bx1)
        y1 = max(y1 - pad_px, by1)
        x2 = min(x2 + pad_px, bx2)
        y2 = min(y2 + pad_px, by2)
        return crop

    crop = crop_in_border(crop)

    c_info = props_for_contours(contours, edges)
    changed = False
    for c in c_info:
        this_crop = c['x1'], c['y1'], c['x2'], c['y2']
        this_area = crop_area(this_crop)
        int_area = crop_area(intersect_crops(crop, this_crop))
        new_crop = crop_in_border(union_crops(crop, this_crop))
        if 0 < int_area < this_area and crop != new_crop:
            print('%s -> %s' % (str(crop), str(new_crop)))
            changed = True
            crop = new_crop

    if changed:
        return pad_crop(crop, contours, edges, border_contour, pad_px)
    else:
        return crop


def downscale_image(im, max_dim=2048):
    """Shrink im until its longest dimension is <= max_dim.
    Returns new_image, scale (where scale <= 1).
    """
    a = im.shape[0]
    b = im.shape[1]
    if max(a, b) <= max_dim:
        return 1.0, im

    scale = 1.0 * max_dim / max(a, b)
    dim = (int(a * scale), int(b * scale))
    new_im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    
    return scale, new_im


def process_image(path, out_path):
    orig_im = Image.open(path)
    im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    scale, im = downscale_image(im)

    edges = cv2.Canny(im, 100, 200)

    # TODO: dilate image _before_ finding a border. This is crazy sensitive!
    _, contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    borders = find_border_components(contours, edges)
    borders.sort(key=lambda i_x1_y1_x2_y2: (i_x1_y1_x2_y2[3] - i_x1_y1_x2_y2[1]) * (i_x1_y1_x2_y2[4] - i_x1_y1_x2_y2[2]))

    border_contour = None
    if len(borders):
        border_contour = contours[borders[0][0]]
        edges = remove_border(border_contour, edges)

    edges = 255 * (edges > 0).astype(np.uint8)

    # Remove ~1px borders using a rank filter.
    maxed_rows = rank_filter(edges, -5, size=(1, 20))
    maxed_cols = rank_filter(edges, -5, size=(20, 1))
    debordered = np.minimum(np.minimum(edges, maxed_rows), maxed_cols)
    edges = debordered

    contours = find_components(edges)
    if len(contours) == 0:
        print('%s -> (no text!)' % path)
        return

    crop = find_optimal_components_subset(contours, edges)
    crop = pad_crop(crop, contours, edges, border_contour)

    crop = [int(x / scale) for x in crop]  # upscale to the original image size.

    # draw and show cropped rectangle area in the original image
    rgb_im = orig_im.convert('RGB')
    draw = ImageDraw.Draw(rgb_im)
    draw.rectangle(crop, outline='red')
    rgb_im.show()

    text_im = orig_im.crop(crop)
    text_im.show()
    text_im.save(out_path)
    print('%s -> %s' % (path, out_path))


if __name__ == '__main__':
    # path = 'images/text.jpg'
    path = 'images/scannedImage.png'
    out_path = 'croppedImage.png'
    try:
        process_image(path, out_path)
    except Exception as e:
        print('%s %s' % (path, e))

images/scannedImage.png not enough values to unpack (expected 3, got 2)
